<a href="https://colab.research.google.com/github/sahilbaishya/HandsOnML/blob/main/Training_Deep_Neural_Networks_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Xavier Garlot and He Initialization

In [32]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

# Nonsaturating Activation Functions

In [33]:
[m for m in dir(keras.activations) if not m.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'gelu',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'swish',
 'tanh']

In [34]:
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

## Neural Network on Fashion MNIST with Leaky Relu

In [35]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [36]:
X_train_full = X_train_full/255.0
X_test = X_test/255.0

In [37]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [15]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [16]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [17]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 1.2848 - accuracy: 0.6095 - val_loss: 0.8754 - val_accuracy: 0.7234
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.7907 - accuracy: 0.7439 - val_loss: 0.7020 - val_accuracy: 0.7750
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6768 - accuracy: 0.7792 - val_loss: 0.6347 - val_accuracy: 0.7940
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.6179 - accuracy: 0.7973 - val_loss: 0.5824 - val_accuracy: 0.8114
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5804 - accuracy: 0.8086 - val_loss: 0.5528 - val_accuracy: 0.8214
Epoch 6/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5535 - accuracy: 0.8157 - val_loss: 0.5306 - val_accuracy: 0.8272
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5330 - accuracy: 0.8211 - val_loss: 0.5115 - val_accuracy:

## Using PReLU

In [19]:
tf.random.set_seed(42)
np.random.seed(42)

model = tf.keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.Dense(300, kernel_initializer = 'he_normal'),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer = 'he_normal'),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation= 'softmax')

])

In [20]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = keras.optimizers.SGD(learning_rate = 1e-3),
              metrics = ['accuracy'])

In [22]:
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 10s 5ms/step - loss: 1.2585 - accuracy: 0.6247 - val_loss: 0.8584 - val_accuracy: 0.7274
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.7770 - accuracy: 0.7521 - val_loss: 0.6922 - val_accuracy: 0.7788
Epoch 3/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.6674 - accuracy: 0.7817 - val_loss: 0.6281 - val_accuracy: 0.7940
Epoch 4/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6101 - accuracy: 0.7987 - val_loss: 0.5755 - val_accuracy: 0.8140
Epoch 5/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5732 - accuracy: 0.8094 - val_loss: 0.5443 - val_accuracy: 0.8232
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5461 - accuracy: 0.8158 - val_loss: 0.5226 - val_accuracy: 0.8272
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5255 - accuracy: 0.8221 - val_loss: 0.5035 - val_accurac

## Using SELU

In [23]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[28, 28]))
model.add(tf.keras.layers.Dense(300, activation = 'selu', kernel_initializer = 'lecun_normal'))
for i in range (99):
  model.add(tf.keras.layers.Dense(100, activation = 'selu', kernel_initializer = 'lecun_normal'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

In [24]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [25]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [26]:
history = model.fit(X_train_scaled, y_train, epochs = 10, validation_data = (X_valid_scaled, y_valid))

Epoch 1/10
1719/1719 [==============================] - 60s 30ms/step - loss: 1.0369 - accuracy: 0.6260 - val_loss: 0.7451 - val_accuracy: 0.7434
Epoch 2/10
1719/1719 [==============================] - 65s 38ms/step - loss: 0.6816 - accuracy: 0.7564 - val_loss: 0.5937 - val_accuracy: 0.7840
Epoch 3/10
1719/1719 [==============================] - 44s 25ms/step - loss: 0.5746 - accuracy: 0.7923 - val_loss: 0.6118 - val_accuracy: 0.7808
Epoch 4/10
1719/1719 [==============================] - 43s 25ms/step - loss: 0.5280 - accuracy: 0.8156 - val_loss: 0.4874 - val_accuracy: 0.8324
Epoch 5/10
1719/1719 [==============================] - 39s 23ms/step - loss: 0.5005 - accuracy: 0.8250 - val_loss: 0.4738 - val_accuracy: 0.8374
Epoch 6/10
1719/1719 [==============================] - 39s 23ms/step - loss: 0.5312 - accuracy: 0.8177 - val_loss: 0.5038 - val_accuracy: 0.8206
Epoch 7/10
1719/1719 [==============================] - 39s 23ms/step - loss: 0.4612 - accuracy: 0.8394 - val_loss: 0.4560 -

## Using ReLU

In [39]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = [28, 28]))
model.add(tf.keras.layers.Dense(300, activation = 'relu', kernel_initializer='he_normal'))
for i in range (99):
  model.add(tf.keras.layers.Dense(100, activation = 'relu', kernel_initializer = 'he_normal'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

In [40]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [41]:
history = model.fit(X_train_scaled, y_train, epochs = 10, validation_data = (X_valid_scaled, y_valid))

Epoch 1/10
1719/1719 [==============================] - 47s 23ms/step - loss: 1.9265 - accuracy: 0.2575 - val_loss: 1.5594 - val_accuracy: 0.4470
Epoch 2/10
1719/1719 [==============================] - 38s 22ms/step - loss: 1.4516 - accuracy: 0.4275 - val_loss: 0.9955 - val_accuracy: 0.6108
Epoch 3/10
1719/1719 [==============================] - 38s 22ms/step - loss: 0.9730 - accuracy: 0.6164 - val_loss: 0.9882 - val_accuracy: 0.5980
Epoch 4/10
1719/1719 [==============================] - 38s 22ms/step - loss: 0.9954 - accuracy: 0.6157 - val_loss: 0.8221 - val_accuracy: 0.7002
Epoch 5/10
1719/1719 [==============================] - 38s 22ms/step - loss: 0.7749 - accuracy: 0.7164 - val_loss: 0.7096 - val_accuracy: 0.7584
Epoch 6/10
1719/1719 [==============================] - 38s 22ms/step - loss: 0.7771 - accuracy: 0.7198 - val_loss: 0.6924 - val_accuracy: 0.7538
Epoch 7/10
1719/1719 [==============================] - 38s 22ms/step - loss: 0.6946 - accuracy: 0.7540 - val_loss: 0.6372 -

# Batch Normalization

In [42]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [43]:
X_train_full = X_train_full/255.0
X_test = X_test/255.0

In [44]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [51]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = [28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

In [52]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_10 (Flatten)        (None, 784)               0         
                                                                 
 batch_normalization_6 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_422 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_7 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_423 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_8 (Batc  (None, 100)             

In [53]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization_6/gamma:0', True),
 ('batch_normalization_6/beta:0', True),
 ('batch_normalization_6/moving_mean:0', False),
 ('batch_normalization_6/moving_variance:0', False)]

In [55]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3),
    metrics = ['accuracy']
)

In [56]:
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 18s 9ms/step - loss: 0.8495 - accuracy: 0.7171 - val_loss: 0.5650 - val_accuracy: 0.8130
Epoch 2/10
1719/1719 [==============================] - 16s 9ms/step - loss: 0.5696 - accuracy: 0.8035 - val_loss: 0.4829 - val_accuracy: 0.8380
Epoch 3/10
1719/1719 [==============================] - 15s 9ms/step - loss: 0.5123 - accuracy: 0.8217 - val_loss: 0.4470 - val_accuracy: 0.8478
Epoch 4/10
1719/1719 [==============================] - 17s 10ms/step - loss: 0.4765 - accuracy: 0.8338 - val_loss: 0.4240 - val_accuracy: 0.8542
Epoch 5/10
1719/1719 [==============================] - 14s 8ms/step - loss: 0.4514 - accuracy: 0.8410 - val_loss: 0.4096 - val_accuracy: 0.8596
Epoch 6/10
1719/1719 [==============================] - 15s 8ms/step - loss: 0.4341 - accuracy: 0.8469 - val_loss: 0.3971 - val_accuracy: 0.8608
Epoch 7/10
1719/1719 [==============================] - 15s 9ms/step - loss: 0.4208 - accuracy: 0.8510 - val_loss: 0.3877 - val_a

## Applying Batch Normalization Before Activation

In [57]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])


In [58]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])


In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))


# Gradient Clipping